# Aula 05

Sumário

- [Aula 05](#aula-05)
  - [Introdução](#introdução)
  - [Bases de Dados](#bases-de-dados)

## Introdução

Vamos seguir o modelo didático do nosso livro-texto. Vamos simular todo um projeto de Aprendizado de Máquina como se fôssemos um cientista de dados contratado por uma empresa de mercado imobiliário. O passo-a-passo que seguiremos:

1. Analisar o panorama geral.
2. Obter os dados.
3. Identificar e visualizar os dados para obter informações úteis.
4. Preparar os dados para os algoritmos de AM.
5. Selecionar e treinar um modelo.
6. Aperfeiçoar o modelo.
7. Apresentar a solução.
8. Disponibilizar em produção, monitar e fazer a manutenção do sistema.

## Bases de Dados

As `bases de dados` das empresas são privadas, ou seja, restritas para uso interno. Entretanto, existem alguns sites que disponibilizam dezenas a centenas de `bases de dados` abertos, os quais podemos baixar para aprendermos os variados algoritmos de AM. Essas `bases de dados` abertas são utilizadas também como `benchmark` de novos modelos e métodos.

A seguir uma lista de repositórios:

* [UCI Irvine ML Repository](http://archive.ics.uci.edu/)
* [Kaggle](https://www.kaggle.com/datasets)
* [AWS](https://registry.opendata.aws/)

Metaportal de dados

* [Data Portals](http://dataportals.org/)

Páginas com listagem de repositórios

* [Wikipedia](https://en.wikipedia.org/wiki/List_of_datasets_for_machine-learning_research)
* [Quora](https://www.quora.com/Where-can-I-find-large-datasets-open-to-the-public)
* [Reddit](https://www.reddit.com/r/datasets)

### A base de dados que será utilizada no nosso exemplo

Vamos seguir com o livro e utilizar a mesma base de dados. O `github` do autor é [Ageron](https://github.com/ageron). Os códigos utilizados por ele para a última versão do seu livro é a [Handson ML3](https://github.com/ageron/handson-ml3/tree/main). A base de dados que vamos utilizar está disponível [nesta página](https://github.com/ageron/data/tree/main/housing).

Para utilizar essa base você tem as seguintes opções:

1. Baixar (ou clonar) o repositório [data](https://github.com/ageron/data/tree/main), e depois pegar somente a base que você quer. Esse caso é o recomendado se você quer pegar logo todas as bases presentes no repositório.
2. Caso você esteja usando `Linux`, dá para seguir o passo-a-passo mostrado na célula 04 [deste arquivo](https://github.com/ageron/handson-ml3/blob/main/02_end_to_end_machine_learning_project.ipynb).
3. Baixar os arquivos `raw` e `md` isoladamente.